# Initializing the System State

## Overview

### Questions

* How do I place particles in the initial condition?
* What units does HOOMD-blue use?

### Objectives

* Describe the basic properties of the **initial condition**, including particle **position**, **orientation**, **type** and the **periodic box**.
* Briefly describe HOOMD-blue's system of **units**.
* Demonstrate writing a system to a **GSD** file.
* Show how to initialize **Simulation state** from a **GSD** file.

## Boilerplate code

In [ ]:
import hoomd
import math
import itertools
import numpy

The `render` function in the next (hidden) cell will render the **initial condition** using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=250, h=250)

def render(position, L):
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=len(position),
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=100)

## Components of the system state

You need to initialize the system **state** before you can run a simulation.
The **initial condition** describes the the **position** and **orientation** of every particle in the system and the **periodic box** at the start of the simulation.

The hard sphere system self-assembles the the fcc structure.
Self-assembly is a process where particles will organize themselves into an ordered structure at equilibrium.
Most self-assembly studies run simulations of many thousands of particles for tens of hours.
To keep this tutorial short, it simulations a small number of particles commensurate with the *fcc* structure (`4 * m**3`, where *m* is an integer).

In [ ]:
m = 5
N_particles = 4 * m**3

## Placing particles

In hard particle Monte Carlo, valid particle configurations have no overlaps.
The particle in this tutorial sits is a sphere of diameter 1, so place particles a little bit further than that apart on a *K*x*K*x*K* simple cubic lattice of width *L*.
The following sections of this tutorial will demonstrate how to randomize this ordered configuration and compress it to a higher density.

In [ ]:
spacing = 1.1
K = math.ceil(N_particles**(1/3))
L = K * spacing

In HOOMD, particle **positions** must be placed inside a **periodic box**.
Cubic boxes range from -*L*/2 to *L*/2.
Use *itertools* and *numpy* to place the positions on the lattice in this range.

In [ ]:
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
print(position[0:4])

Filter this list down to *N* particles because `K*K*K >= N_particles` .

In [ ]:
position = position[0:N_particles]

Here is what the system looks like now:

In [ ]:
render(position, L)

## Units

HOOMD-blue does not adopt any particular real system of **units**. 
Instead, HOOMD-blue uses an *internally self-consistent* system of **units** and is compatible with *many systems of units*.
For example: if you select the units of *meter*, *Joule*, and *kilogram* for length, energy and mass then the units of force will be *Newtons* and velocity will be *meters/second*.
A popular system of units for nano-scale systems is *nanometers*, *kilojoules/mol*, and *atomic mass units*.

In HPMC, the *primary* **unit** is that of *length*.
*Mass* exists, but is factored out of the partition function and does not enter into the simulation.
The scale of *energy* is irrelevant in athermal HPMC systems where overlapping energies are infinite and valid configurations have zero potential energy.
However, *energy* does appear implicitly in **derived units** like $[\mathrm{pressure}] = \left(\frac{\mathrm{[energy]}}{\mathrm{[length]}^3}\right)$.
In HPMC, $kT$ is assumed to be 1 *energy*.
You can convert a pressure $P$ in HPMC's units to a non-dimensional value with $P^* = \frac{P\sigma^3}{kT}$ where $\sigma$ is a representative length in your system, such as the particle's diameter or edge length.

## Writing the configuration to the file system

[**GSD**](https://gsd.readthedocs.io/) files store the **periodic box**, particle **positions**, **orientations**,  and other properties of the state.
Use the **GSD** Python package to write this file.

In [ ]:
import gsd.hoomd

The **Snapshot** object stores the state of the system.

In [ ]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles
snapshot.particles.position = position

Each particle also has a **type**.
In HPMC, each **type** has its own shape parameters.
There is a single **type** in this tutorial, so set a **type id** of 0 for all particles (type ids are 0-indexed):

In [ ]:
snapshot.particles.typeid = [0]*N_particles

Every particle **type** needs a **name**.
Names can be any string.
HOOMD-blue uses the type names to match the parameters you specify in **operations** with the type names present in the **state**.
Give the name `'sphere'` to the type id 0:

In [ ]:
snapshot.particles.types = ['sphere']

**GSD** represents boxes with a 6-element array. Three box lengths *L_x*, *L_y*, *L_z*, and 3 tilt factors. Set equal box lengths 0 tilt factors to define a cubic box.

In [ ]:
snapshot.configuration.box = [L, L, L, 0, 0, 0]

Write this snapshot to `lattice.gsd`:

In [ ]:
with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
    f.append(snapshot)

## Initializing a Simulation

You can use the file to initialize the **Simulation state**.
First, create a **Simulation**:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)

Then use the `create_state_from_gsd` factory function to read in the GSD file and populate the **state** with the particles and box from the file:

In [ ]:
sim.create_state_from_gsd(filename='lattice.gsd')

The next section in this tutorial will show you how to use HPMC to randomize the particle positions and orientations.

[Previous section](02-Performing-Hard-Particle-Monte-Carlo-Simulations.ipynb). [Next section](04-Randomizing-the-System.ipynb).